In [ ]:
# (Run this / Run all) Import libraries
import os
from datetime import datetime
import zipfile

In [ ]:
# Enable functions to zip output files

def get_file_list(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.endswith('.zip'):
                file_path = os.path.join(root, file)
                file_list.append(file_path)
    return file_list

def create_zip(file_list, zip_name, output_directory, max_size):
    zip_count = 1
    current_zip_size = 0  # Track current zip file size
    current_file_list = []
    for file in file_list:
        file_size = os.path.getsize(file)
        if current_zip_size + file_size > max_size:
            create_zip_file(current_file_list, zip_name, output_directory, zip_count)
            current_zip_size = 0
            current_file_list = []
            zip_count += 1

        current_file_list.append(file)
        current_zip_size += file_size

    if current_file_list:
        create_zip_file(current_file_list, zip_name, output_directory, zip_count)

def create_zip_file(file_list, zip_name, output_directory, zip_count):
    current_zip_name = f"{zip_name}_part{zip_count}.zip"
    current_zip_path = os.path.join(output_directory, current_zip_name)
    with zipfile.ZipFile(current_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in file_list:
            arcname = os.path.relpath(file, output_directory)
            zipf.write(file, arcname)

In [ ]:
### (Run this) Zip all files and folders in /outputs except zip files
# Configurations
output_path = '/notebooks/stable-diffusion-webui/outputs'
max_file_size = 250 * 1024 * 1024  # Maximum file size in bytes

file_list = get_file_list(output_path)
output_name = f"outputs_{datetime.now().strftime('%Y%m%d%H%M%S')}"

try:
    create_zip(file_list, output_name, output_path, max_file_size)
    print("Success: All files have been zipped successfully.")
except Exception as e:
    print(f"Error occurred: {str(e)}")

In [ ]:
### (Alternative for notebook) Uncomment and run to export all folders and files inside outputs
# (Alternative for terminal) Uncomment, then remove '%' and '!'
# %cd /notebooks/stable-diffusion-webui/outputs
# !zip -r $("outputs_" + date +"%Y%m%d%H%M%S").zip .